<h1><span style="color:red">Base de Datos Relacionales para una Red Social</span></h1>

<p><h3><b>Nota:</b> Este proyecto aún no esta términado, faltan algunas funcionalidades y tambien le falta ser optimizado</h3></p>

<h2><span style="color:blue">1. Descripción del Proyecto</span></h2>
<p>En este proyecto crearemos una serie de clases de python para crear una simulación de base de datos para una red social basada en texto que posea las siguientes características:
<ul>
 <li>Almacene la información personal de los usuarios al momento de estos registrarse en la red social.
 <li>Genere un registro que permita relacionar seguidores con seguidos dentro de la red social.
 <li>Almacene las públicaciones en formato de texto de los usuarios y les asigne un registro único que relacione        publicación y publicador.
 <li>Lleve una lista de que usuarios hicieron like a que publicaciones.
 <li>Que las tablas donde se almacenacene toda esta información solo puedan ser creadas o eliminadas por un administrador mediante una contraseña.
 <li>Que las bases de datos actualicen su información en caso algun usuario deje de seguir a otro o deje de dar like a alguna publicacion.
 <li>La base de datos debe estar almacenada en el ordenador local.
</ul>
Para todo lo anterior utilizaremos unicamente el módulo de <a HREF="https://docs.python.org/es/3/library/sqlite3.html">sqlite3</a> que nos permitira crear bases de datos relacionales utilizando el lenguaje SQL mediante la librería <a HREF="https://www.sqlite.org/index.html">SQLite</a>, haciendo asi que podamos trabajar dentro de un notebook de Jupyter Lab. 
</p>

In [1]:
# %load_ext sql <-- Este se usaria si no tenemos dicha extensión instalada.
import sqlite3

# Por ahora, al estar trabajando de forma local, necesitaremos de una ruta en el ordenador:
path='C:\\Users\\User\\Desktop\\Portafolio\\DataSets\\bases_de_datos_relacionales_i\\red_social_database.db'

<h2><span style="color:blue">2. Creando las Bases de Datos</span></h2>
<p>Para cumplir con las condiciones que se nos piden, primero debemos crear la estructura de la Base de Datos, para ello sera necesario crear 4 tablas con las siguientes caracteristicas:
<lu>
<li><b>Tabla de Usuarios:</b> Que almacenara todas la información del usuario para que se registre, a saber: Su Seudónimo (que debe ser único), nombre (primero y segundo), teléfono y correo electrónico, además de llevar registro de cuando se creo la cuenta. Por razones obvias esta tabla no dependera de ninguna otra.
<li><b>Tabla de Seguidores:</b> Que almacene el quien sigue a quien y donde evidentemente no se puede seguir a uno mismo o lo que es igual debe poseer: la identificación del usuario seguidor y la identificación del usuario seguido. Por lo anterior esta tabla solo dependera de la Tabla de Usuarios.
<li><b>Tabla de Publicaciones:</b> Esta tabla debe almacenar: la identificación de la publicación, el texto de la publicación, los likes de la publicación y por supuesto la fecha en la que se hizo la publicación. De esta manera, se relacionara la tabla con la Tabla de Usuarios y la Tabla de Likes.
<li><b>Tabla de likes:</b> Esta tabla debe contener un registro de quien realizo el like y a cual publicación la hizo. Se relacionara entonces con la Tabla de Usuarios y la Tabla de Publicaciones. 
</lu>
Con todo lo anterior tendremos que la estructura de nuestra base de datos es de la siguiente manera:
</p>
<figure>
<center>
    <img src="https://raw.githubusercontent.com/juanetoboneto/Portafolio/DataSets/bases_de_datos_relacionales_i/Diagrama_de_base_de_datos_i.jpg" width="600" height="400" align="center" />
</center>
</figure>
<p>
Teniendo en cuenta lo que se nos pide, la creación y eliminación de las tablas las realizaremos dentro de la clase de los <I>administradores</I> como sigue:
</p>

In [2]:
#Clase con las funcionalidades de Administrador :
class administrador:
      
 #Función para verificar la contraseña de administrador:    
 contraseña = "contraseñaxd" 
 def verificar_contraseña(contraseña_ingresada):
    return contraseña_ingresada == administrador.contraseña
    
 #Funcionalidades de administrador para la tabla de usuarios:

 def crear_tabla_usuarios(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):    
     try:
      conexion = sqlite3.connect(path)
      cursor = conexion.cursor()
      cursor.execute("""CREATE TABLE IF NOT EXISTS tabla_usuarios (
                        identificacion_usuario INTEGER PRIMARY KEY,
                        seudonimo_usuario VARCHAR(20) NOT NULL UNIQUE,
                        correo_electronico VARCHAR(40) NOT NULL UNIQUE,
                        primer_nombre VARCHAR(80) NOT NULL,
                        segundo_nombre VARCHAR(80) NOT NULL,
                        telefono VARCHAR(12) UNIQUE,
                        creado_el TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
                     )""")
      conexion.close()
      print("Se pudo crear la Tabla de Usuarios correctamente.")   
     except Exception as e:
            print(f"No se pudo crear la tabla de usuarios. Error: {e}")   
    else:
        print("Contraseña incorrecta")
    
 def eliminar_tabla_usuarios(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
     try:
      conexion = sqlite3.connect(path)
      cursor = conexion.cursor()
      cursor.execute("""DROP TABLE IF EXISTS tabla_usuarios""")
      conexion.close()
      print("Se pudo eliminar la Tabla de Usuarios correctamente.") 
     except Exception as e:
            print(f"No se pudo eliminar la tabla de usuarios. Error: {e}")
    else:
        print("Contraseña incorrecta")
    
 #Funcionalidades de administrador para la tabla de seguidores:

 #Nota: esta tabla tiene la condición de que nadie puede seguirse a sí mismo
 def crear_tabla_seguidores(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         conexion = sqlite3.connect(path)
         cursor = conexion.cursor()
         cursor.execute("""CREATE TABLE IF NOT EXISTS tabla_seguidores (
                           identificacion_seguidor INT NOT NULL,
                           identificacion_seguido INT NOT NULL,
                           FOREIGN KEY (identificacion_seguidor) REFERENCES tabla_usuarios(identificacion_usuario),
                           FOREIGN KEY (identificacion_seguido) REFERENCES tabla_usuarios(identificacion_usuario),
                           PRIMARY KEY (identificacion_seguidor, identificacion_seguido) 
                        )""")
         conexion.close()
         print("Se pudo crear la Tabla de Seguidores correctamente.")
        except Exception as e:
            print(f"No se pudo crear la Tabla de de Seguidores. Error: {e}")
    else:
        print("Contraseña incorrecta") 

 def eliminar_tabla_seguidores(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         conexion = sqlite3.connect(path)
         cursor = conexion.cursor()
         cursor.execute("""DROP TABLE IF EXISTS tabla_seguidores""")
         conexion.close()
         print("Se pudo eliminar la Tabla de Seguidores correctamente.")
        except Exception as e:
            print(f"No se pudo eliminar la Tabla de Seguidores. Error: {e}")
    else:
        print("Contraseña incorrecta")
        
 #Funcionalidades de administrador para la tabla de publicaciones:
 
 def crear_tabla_publicaciones(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         conexion = sqlite3.connect(path)
         cursor = conexion.cursor()
         cursor.execute("""CREATE TABLE IF NOT EXISTS tabla_publicaciones (
                           identificacion_publicacion INTEGER PRIMARY KEY,
                           identificacion_usuario INT NOT NULL,
                           texto_publicacion VARCHAR(12000) NOT NULL,
                           numero_likes INT DEFAULT 0,
                           creado_en TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
                           FOREIGN KEY (identificacion_usuario) REFERENCES tabla_usuarios(identificacion_usuario)  
                        )""")
         conexion.close()
         print("Se pudo crear la Tabla de Publicaciones correctamente.")
        except Exception as e:
            print(f"No se pudo crear la Tabla de Publicaciones. Error:{e}")
    else:
        print("Contraseña incorrecta")
            
 def eliminar_tabla_publicaciones(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         conexion = sqlite3.connect(path)
         cursor = conexion.cursor()
         cursor.execute("""DROP TABLE IF EXISTS tabla_publicaciones""")
         conexion.close()
         print("Se pudo eliminar la Tabla de Publicaciones correctamente.")
        except Exception as e:
            print(f"No se pudo eliminar la Tabla de Publicaciones. Error:{e}")
    else:
        print("Contraseña incorrecta")
            
 #Funcionalidades de administrador para la tabla de "likes":

 def crear_tabla_likes(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         conexion = sqlite3.connect(path)
         cursor = conexion.cursor()
         cursor.execute("""CREATE TABLE IF NOT EXISTS tabla_likes (
                           identificacion_usuario INT NOT NULL,
                           identificacion_publicacion INT NOT NULL,
                           FOREIGN KEY (identificacion_usuario) REFERENCES tabla_usuarios(identificacion_usuario),
                           FOREIGN KEY (identificacion_publicacion) REFERENCES tabla_publicaciones(identificacion_publicacion),
                           PRIMARY KEY (identificacion_usuario, identificacion_publicacion)
                        )""")
         conexion.close()
         print("Se pudo crear la Tabla de Likes correctamente.")
        except Exception as e:
            print(f"No se pudo crear la Tabla de Likes. Error:{e}")
    else:
        print("Contraseña incorrecta")

 def eliminar_tabla_likes(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         conexion = sqlite3.connect(path)
         cursor = conexion.cursor()
         cursor.execute("""DROP TABLE IF EXISTS tabla_likes""")
         conexion.close()
         print("Se pudo eliminar la Tabla de Likes correctamente.")
        except Exception as e:
            print(f"No se pudo eliminar la Tabla de Likes. Error:{e}")
    else:
        print("Contraseña incorrecta")
        
 # Operaciones con multiples tablas:

 def crear_todas_las_tablas(contraseña_ingresada):
   administrador.crear_tabla_usuarios(contraseña_ingresada)
   administrador.crear_tabla_seguidores(contraseña_ingresada)
   administrador.crear_tabla_publicaciones(contraseña_ingresada)
   administrador.crear_tabla_likes(contraseña_ingresada)

 def eliminar_todas_las_tablas(contraseña_ingresada):
  administrador.eliminar_tabla_usuarios(contraseña_ingresada)
  administrador.eliminar_tabla_seguidores(contraseña_ingresada)
  administrador.eliminar_tabla_publicaciones(contraseña_ingresada)
  administrador.eliminar_tabla_likes(contraseña_ingresada)

<h2><span style="color:blue">3. Creando las Funcionalidades de Usuario</span></h2>
<p>
 La siguiente clase debe poseer todas las funciones que solo estan disponibles para las personas registradas en la base de datos, esto quiere decir que necesariamente estan registrados en la red social, o lo que es igual existen valores de ellos en la Tabla de Usuarios. Por las condiciones que se nos fueron dadas deben existir al menos 7 funciones (yo tambien planeo agregar algunas otras) a saber: registrar una cuenta, eliminar dicha cuenta, seguir otras cuentas, dejar de seguir dichas cuentas, publicar y eliminar cuentas, por último dar like. Todas esas funciones agregan o eliminan valores en la estructura de datos que creamos antes, para ello crearemos la clase para los Usuarios:
</p>

In [3]:
# Creemos una clase que sintetize todas las funcionalidades para el usuario:
class usuario:
 
 # Utilizaremos el instanciamiento de un usuario en la clase como su registro en la red social: 
 def __init__(self,seudonimo_usuario, correo_electronico, primer_nombre, segundo_nombre, telefono):
    
    self.seudonimo_usuario = seudonimo_usuario
    self.correo_electronico = correo_electronico
    self.primer_nombre = primer_nombre
    self.segundo_nombre = segundo_nombre
    self.telefono = telefono

    conexion = sqlite3.connect(path)
    cursor = conexion.cursor()
    cursor.execute("""INSERT INTO tabla_usuarios (seudonimo_usuario, correo_electronico, primer_nombre, segundo_nombre, telefono)
                   VALUES(?,?,?,?,?)""",
                   (self.seudonimo_usuario, self.correo_electronico, self.primer_nombre, self.segundo_nombre, self.telefono))
    conexion.commit()
    conexion.close()
    
    print(f"Su cuenta {seudonimo_usuario} ha sido creada correctamente.")
    
 # Funcionalidades de Usuario excusivas para la tabla de Usuarios:
    
 def buscar_usuario(usuario, usuario_buscado):
    conexion = sqlite3.connect(path)
    cursor = conexion.cursor()
    cursor.execute("SELECT seudonimo_usuario,primer_nombre,segundo_nombre FROM tabla_usuarios WHERE seudonimo_usuario=?", (usuario_buscado,))
    print(cursor.fetchall())
    conexion.close()
    
 def eliminar_usuario(usuario):
    conexion = sqlite3.connect(path)
    cursor = conexion.cursor()
    cursor.execute("""DELETE  FROM tabla_usuarios WHERE seudonimo_usuario = ?""", (usuario.seudonimo_usuario,))
    conexion.commit()
    conexion.close()
 
 # Funcionalidades de la tabla de seguidores:

 # Estas funcionalidades son algo más complicadas debido a que necesitaremos realizar queris por orden a la tabla de usuarios
 # para poder luego guardarlas en la tabla de seguidores:

 def seguir_usuario(usuario, seguir):
  try:
   # realizando la query del ID del seguidor:     
   conexion_1 = sqlite3.connect(path)
   cursor_1 = conexion_1.cursor()
   cursor_1.execute("SELECT identificacion_usuario FROM tabla_usuarios WHERE seudonimo_usuario=?",
                   (usuario.seudonimo_usuario, ))
   relacion_1 = cursor_1.fetchone()
   conexion_1.close()   
   x_1 = str(relacion_1[0])
   # realizando la query del ID del seguido:
   conexion_2 = sqlite3.connect(path)
   cursor_2 = conexion_2.cursor()
   cursor_2.execute("SELECT identificacion_usuario FROM tabla_usuarios WHERE seudonimo_usuario=?",
                   (seguir, ))
   relacion_2 = cursor_2.fetchone()
   conexion_2.close() 
   x_2 = str(relacion_2[0])
   # guardando dicha relacion en la tabla de seguidores:
   conexion_3 = sqlite3.connect(path)
   cursor_3 = conexion_3.cursor()
   cursor_3.execute("INSERT INTO tabla_seguidores (identificacion_seguidor, identificacion_seguido) VALUES(?,?)", 
                    (x_1, x_2))
   conexion_3.commit()
   conexion_3.close()
   print(f"{usuario.seudonimo_usuario} ahora sigue a {seguir}")
  except Exception as e:
    print("Tu cuenta o la de la persona que intentas seguir no existen.")
    
 def dejar_de_seguir_usuario(usuario, dejar_de_seguir):
    try:
     # realizando la query del ID del seguidor:     
     conexion_1 = sqlite3.connect(path)
     cursor_1 = conexion_1.cursor()
     cursor_1.execute("SELECT identificacion_usuario FROM tabla_usuarios WHERE seudonimo_usuario=?",
                   (usuario.seudonimo_usuario, ))
     relacion_1 = cursor_1.fetchone()
     conexion_1.close()   
     x_1 = str(relacion_1[0])
     # realizando la query del ID del seguido:
     conexion_2 = sqlite3.connect(path)
     cursor_2 = conexion_2.cursor()
     cursor_2.execute("SELECT identificacion_usuario FROM tabla_usuarios WHERE seudonimo_usuario=?",
                   (dejar_de_seguir, ))
     relacion_2 = cursor_2.fetchone()
     conexion_2.close() 
     x_2 = str(relacion_2[0])
     # eliminando dicha relación de la tabla de seguidores:
     conexion_3 = sqlite3.connect(path)
     cursor_3 = conexion_3.cursor()
     cursor_3.execute("""DELETE FROM tabla_seguidores
                         WHERE (identificacion_seguidor, identificacion_seguido) = (?, ?)""", (x_1, x_2))
     conexion_3.commit()
     conexion_3.close()
     print(f"{usuario.seudonimo_usuario} dejo de seguir a {dejar_de_seguir}")
    except Exception as e:
        print(f"No se pudo anular el seguido: Error {e}")
        
 # Funcionalidades de usuario para la Tabla de Publicaciones:

 # Podriamos utilizar una condición "if" para asegurarnos que la entrada de las publicaciones sea necesariamente
 # un string, sin embargo esto extenderia innecesariamente el código, solo para que a efectos prácticos se tenga 
 # el mismo resultado que introducir una variable "normal".
    
 def publicar_comentario(usuario, publicacion):
    try:
     conexion_1 = sqlite3.connect(path)
     cursor_1 = conexion_1.cursor()
     cursor_1.execute("SELECT identificacion_usuario FROM tabla_usuarios WHERE seudonimo_usuario=?",
                   (usuario.seudonimo_usuario, ))
     id_usuario = cursor_1.fetchone()
     conexion_1.close()  
     id_usuario = str(id_usuario[0])
     conexion_2 = sqlite3.connect(path)
     cursor_2 = conexion_2.cursor()
     cursor_2.execute("INSERT INTO tabla_publicaciones (identificacion_usuario, texto_publicacion) VALUES(?,?)",
                    (id_usuario, publicacion))
     conexion_2.commit()
     conexion_2.close()
     print("Su comentario a sido publicado con éxito.")
    except Exception as e:
     print(f"Tu comentario no pudo ser publicado. Error:{e}")
  
 # Para la funcion de eliminar publicacion necesitaremos realizar 2 querys, una que nos de el ID del usuario y
 # otra que nos permita eliminar su comentario:
 def eliminar_publicacion(usuario, publicacion):
  try:     
   conexion_1 = sqlite3.connect(path)
   cursor_1 = conexion_1.cursor()
   cursor_1.execute("""SELECT identificacion_usuario FROM tabla_usuarios 
                    WHERE seudonimo_usuario=?""", (usuario.seudonimo_usuario, ))
   id_usuario = cursor_1.fetchone()
   conexion_1.close()
   id_usuario = str(id_usuario[0])
   conexion_2 = sqlite3.connect(path)
   cursor_2 = conexion_2.cursor()
   cursor_2.execute("""DELETE FROM tabla_publicaciones 
                    WHERE (identificacion_usuario, texto_publicacion) = (?,?)""",(id_usuario, publicacion))
   conexion_2.commit()
   conexion_2.close()
   print("Su publicación a sido eliminado con éxito.")
  except Exception as e:
   print(f"Tu publicación no pudo ser eliminada. Error:{e}")
    
    
 # Funcionalidades de usuario para la Tabla de Likes:

 def dar_like(usuario, publicador, publicacion):
  try:     
   conexion_1 = sqlite3.connect(path)
   cursor_1 = conexion_1.cursor()
   cursor_1.execute("""SELECT identificacion_usuario FROM tabla_usuarios 
                    WHERE seudonimo_usuario=?""", (publicador, ))
   id_publicador = cursor_1.fetchone()
   conexion_1.close()
   id_publicador = str(id_publicador[0])
   conexion_2 = sqlite3.connect(path)
   cursor_2 = conexion_2.cursor()
   cursor_2.execute("""SELECT identificacion_publicacion 
                       FROM tabla_publicaciones 
                       WHERE (identificacion_usuario, texto_publicacion)=(?,?)""", (id_publicador, publicacion))
   id_publicacion = cursor_2.fetchone()
   conexion_2.close()
   id_publicacion = str(id_publicacion[0])
   conexion_3 = sqlite3.connect(path)
   cursor_3 = conexion_3.cursor()
   cursor_3.execute("""INSERT INTO tabla_likes (identificacion_usuario, identificacion_publicacion) VALUES(?,?)""",
                    (id_publicador, id_publicacion))
   conexion_3.commit()
   conexion_3.close()
   print(f"A {usuario.seudonimo_usuario} le a gustado {publicacion}")
  except Exception as e:
   print(f"No se pudo dar like. Error:{e}") 

<h2><span style="color:blue">3. Probando nuestras Bases de Datos</span></h2>
<p>Por último veamos como se manejan nuestras bases de datos relacionales.
 Empecemos intentando crear las tablas sin la contraseña de administrador:</p>

In [4]:
administrador.crear_tabla_usuarios("No la contraseña correcta")

Contraseña incorrecta


<p>Ahora veamos como se pueden eliminar y crear toda la estructura de bases de datos a la vez, siendo un administrador con la contraseña correcta:</p>

In [5]:
administrador.crear_todas_las_tablas("contraseñaxd")

Se pudo crear la Tabla de Usuarios correctamente.
Se pudo crear la Tabla de Seguidores correctamente.
Se pudo crear la Tabla de Publicaciones correctamente.
Se pudo crear la Tabla de Likes correctamente.


<p>Tambien, creemos un ejemplo de 2 Usuarios que se instancian como objetos de dicha clase. Registrando, a su vez, dichos datos en la Tabla de usuarios:</p>

In [6]:
juan = usuario("juan", "juanes@gmail.com", "juan", "esteban", "3014415299")
jose = usuario("jose", "josefo@gmail.com", "jose", "manuel", "3113739070")

Su cuenta juan ha sido creada correctamente.
Su cuenta jose ha sido creada correctamente.


Intentemos ahora buscar un usuario desde la perspectiva de alguien no registrado:

In [7]:
alguien.buscar_usuario("juan")

NameError: name 'alguien' is not defined

En cambio, si lo hacemos desde alguien ya instanciado:

In [8]:
jose.buscar_usuario("juan")

[('juan', 'juan', 'esteban')]


<p>Veamos en cambio, como se registran los seguidores y el dejar de seguir, mediante la correspondiente función:</p>

In [9]:
juan.seguir_usuario("jose")
juan.dejar_de_seguir_usuario("jose")

juan ahora sigue a jose
juan dejo de seguir a jose


<p>Con el ejemplo anterior veamos como se registran la publicacion de Juan y como Jose le da like al mismo:</p>

In [10]:
juan.publicar_comentario("Hola gente, este es mi primer comentario siganme en esta red social plz")
jose.dar_like("juan", "Hola gente, este es mi primer comentario siganme en esta red social plz")

Su comentario a sido publicado con éxito.
A jose le a gustado Hola gente, este es mi primer comentario siganme en esta red social plz


Veamos como Juan se arrepiente de la publicación que hizo y decide eliminarla:

In [11]:
juan.eliminar_publicacion("Hola gente, este es mi primer comentario siganme en esta red social plz")

Su publicación a sido eliminado con éxito.


<p>Por último eliminemos toda la estructura de datos que hicimos:</p>

In [12]:
administrador.eliminar_todas_las_tablas("contraseñaxd")

Se pudo eliminar la Tabla de Usuarios correctamente.
Se pudo eliminar la Tabla de Seguidores correctamente.
Se pudo eliminar la Tabla de Publicaciones correctamente.
Se pudo eliminar la Tabla de Likes correctamente.
